In [1]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import configparser

# setting the starting url 
url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.0.1.html"

## First testing

In [2]:
# Testing 
# Sends a GET request to the URL to retrieve its content; The Website tryes to redicrect the request to a captcha page, reloading solves the problem
response = requests.get(url_overview_köln, allow_redirects=False)
# assert response.status_code == 200
print(response.status_code)
# Parse results
soup = BeautifulSoup(response.text, 'html.parser')

target_class = "ang_spalte_icons row_click"
elements = soup.find_all('td', class_=target_class)
links=[]
for element in elements:
    links.append(element.find('a').get('href'))
# Extracting all shared apartments
#for link in soup.find_all('a'):
#    print(link.get('href'))
print(links)
print(len(links))


200
['wg-zimmer-in-Koeln-Ehrenfeld.11008948.html', 'wg-zimmer-in-Koeln-Hoehenberg.10926775.html', 'wg-zimmer-in-Koeln-Ehrenfeld.11019202.html', 'wg-zimmer-in-Koeln-Suelz.8508188.html', 'wg-zimmer-in-Koeln-Lindenthal.11019197.html', 'wg-zimmer-in-Koeln-Muelheim.10451742.html', 'wg-zimmer-in-Koeln-Leverkusen-Wiesdorf.8836353.html', 'wg-zimmer-in-Koeln-Neuehrenfeld-Neu-Ossendorf.11017025.html', 'wg-zimmer-in-Koeln-Neustadt-Sued.8242931.html', 'wg-zimmer-in-Koeln-Lindenthal.10733309.html', 'wg-zimmer-in-Koeln-Koeln-Ostheim.10998761.html', 'wg-zimmer-in-Koeln-Braunsfeld.11002555.html', 'wg-zimmer-in-Koeln-Suerth.7363549.html', 'wg-zimmer-in-Koeln.10972581.html', 'wg-zimmer-in-Koeln-Niehl.10974009.html', 'wg-zimmer-in-Koeln-Brueck.10036288.html', 'wg-zimmer-in-Koeln-Ehrenfeld.10967178.html', 'wg-zimmer-in-Koeln-Altstadt.8824542.html', 'wg-zimmer-in-Koeln-Humboldt-Gremberg.10972149.html', 'wg-zimmer-in-Koeln-Nippes.5752668.html']
20


302 => Captcha -->nochmal probieren
200 => Okay

wnen die Klasse auf einer Seite nicht mehr gefunden werden kann, enthält diese keine Angebote mehr

## Getting the Links

In [13]:
page = 0
failed_overview = 0
base_links=[]
campaign_links=[]

# main class for the links
target_class = "truncate_title noprint"

# class for the campaign links
campaign_class = "campaign_click"

last_page = page
cum_fails = 0
while page <= 20:
    # setting the url 
    url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.1."+str(page)+".html"

    # Sends a GET request to the URL to retrieve its content; sometimes the Website tryes to redicrect the request to a captcha page, reloading solves the problem
    response = requests.get(url_overview_köln, allow_redirects=False)
    
    # test response code
    if response.status_code == 302 and failed_overview < 5:
        # try again
        failed_overview += 1 
        cum_fails += 1
        # if needed, add a sleep here
        continue
    elif failed_overview >= 5:
        print("Failed 5 times in a row")
        break
    
    page += 1
    failed_overview = 0

    # Parse results
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting all shared apartments
    elements = soup.find_all('h3', class_ = target_class)

    if len(elements) == 0:
        print("No more elements found")
        break

    
    for element in elements:
        for appartment in element.find_all('a', class_ = campaign_class):
            campaign_links.append(element.find('a').get('href'))
        for appartment in element.find_all('a', class_=''):
            base_links.append(element.find('a').get('href'))

    # just for debugging
    last_page = page -1


# TODO: aufhübschen und in Funktionen packen
print("Base links: ")
print(base_links)

set_base_links = set(base_links)
base_links = list(set_base_links)
print("Anzahl der Base links:" + str(len(base_links)))



print("Campaign links: " )
print(campaign_links)

set_campaign_links = set(campaign_links)
campaign_links = list(set_campaign_links)
print("Anzahl der Campaign links:" + str(len(campaign_links)))
print("Pages: "+str(last_page))
print("Failed: "+str(cum_fails))


Base links: 
['/wg-zimmer-in-Koeln-Zollstock.6544556.html', '/wg-zimmer-in-Koeln-Humboldt-Gremberg.10526631.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.11019208.html', '/wg-zimmer-in-Koeln-Loevenich.2085119.html', '/wg-zimmer-in-Koeln-Ehrenfeld.10713040.html', '/wg-zimmer-in-Koeln-Altstadt-Nord.10924685.html', '/wg-zimmer-in-Koeln-Marienburg.3216416.html', '/wg-zimmer-in-Koeln-Koeln.11005086.html', '/wg-zimmer-in-Koeln-Rondorf.10770754.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.3457932.html', '/wg-zimmer-in-Koeln-Kalk.9751650.html', '/wg-zimmer-in-Koeln-Worringen.7494243.html', '/wg-zimmer-in-Koeln-Humboldt-Gremberg.9090325.html', '/wg-zimmer-in-Koeln-Lindenthal.10792004.html', '/wg-zimmer-in-Koeln-Lindenthal.10998254.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.7764003.html', '/wg-zimmer-in-Koeln-Altstadt-Nord.10433154.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.10081212.html', '/wg-zimmer-in-Koeln-Junkersdorf.5595834.html', '/wg-zimmer-in-Koeln-Junkersdorf.5579974.html', '/wg-zimmer-in-Koeln

## Getting Information for different shared Appartments

nur verifizierte Unternehmen haben die Klasse "campaign clicks" --> sind auch unten als Werbung aufgeführt

verfifizierte Personen nicht



416 Ergebnisse sind okay. Erwartet hätten wir 420 -> ggf. Werbungen?

In [4]:
# TODO: remove wrong links in the camoaign links (asset_id...)



In [18]:
# TODO: für eine Einzelseite die Attribute bestimmen
result_df = pd.DataFrame(columns=['Link', 'Title', 'Size', 'Rent', 'Extra Costs', 'Other Costs', 'Deposit', 'Redemption Agreement', 'Address', 'Available From', 'Available Till', 'Online Since'])

# Testing
test_counter = 0
failed_app_acc = 0

shared_app = 0

while shared_app < len(base_links):
    url_shared_app = "https://www.wg-gesucht.de" + str(base_links[shared_app]) 
    response = requests.get(url_shared_app, allow_redirects=False)
    print(response.status_code)
    # test response code
    if response.status_code == 302 and failed_app_acc < 5:
        # try again
        failed_app_acc += 1 
        cum_fails += 1
        # if needed, add a sleep here
        time.sleep(10)
        continue
    elif failed_app_acc >= 5:
        raise Exception("Failed 5 times in a row")

    if response.status_code == 503:
        print("Service Unavailable. Retrying in 15 seconds")
        time.sleep(15)
        failed_app_acc += 1 
        cum_fails += 1
        continue

    shared_app += 1
    failed_app_acc = 0

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting the title
    title = soup.find('h1', class_='headline headline-detailed-view-title').text
    title = title.replace('\n', '').strip()

    # Extracting the size
    size = soup.find('b', class_='key_fact_value').text
    size = size.replace('\n', '').strip()

    # TODO: Adding the count of people in the flat
    # TODO: Maybe Restructure the replacing of the \n and strip to a function and do it in the df
    # Extracting the cost panel
    cost_panel = soup.find_all('div', class_='panel section_panel')[1]

    rent = cost_panel.find_all('span', class_='section_panel_value')[0].text
    rent = rent.replace('\n', '').strip()

    extra_costs = cost_panel.find_all('span', class_='section_panel_value')[1].text
    extra_costs = extra_costs.replace('\n', '').strip()

    other_costs = cost_panel.find_all('span', class_='section_panel_value')[2].text
    other_costs = other_costs.replace('\n', '').strip()

    deposit = cost_panel.find_all('span', class_='section_panel_value')[3].text
    deposit = deposit.replace('\n', '').strip()

    redemption_agreement = cost_panel.find_all('span', class_='section_panel_value')[4].text
    redemption_agreement = redemption_agreement.replace('\n', '').strip()



    # Address and Availability Panel
    address_panel = soup.find_all('div', class_='panel section_panel')[2]

    address = address_panel.find_all('span', class_='section_panel_detail')[0].text
    address = ' '.join(address.split())

    available_from = address_panel.find_all('span', class_='section_panel_value')[0].text
    available_from = available_from.replace('\n', '').strip()

    # Sometimes the available till is not available, so we need to check if the second span is 'frei bis:' --> if not we need to set it to n.a.
    if address_panel.find_all('span', class_='section_panel_detail')[2].text.strip() == 'frei bis:':
        available_till = address_panel.find_all('span', class_='section_panel_value')[1].text
        available_till = available_till.replace('\n', '').strip()
        online_since = address_panel.find_all('b', class_='noprint')[0].text
        online_since = online_since.replace('\n', '').strip()

    else:
        available_till = "n.a."
        online_since = address_panel.find_all('b', class_='noprint')[0].text
        online_since = online_since.replace('\n', '').strip()


    input_set = {'Link': url_shared_app, 'Title': title, 'Size': size, 'Rent': rent, 'Extra Costs': extra_costs, 'Other Costs': other_costs, 'Deposit': deposit, 'Redemption Agreement': redemption_agreement, 'Address': address, 'Available From': available_from, 'Available Till': available_till, 'Online Since': online_since}
    result_df = result_df._append(input_set, ignore_index=True)

    # Testing
    test_counter += 100
    if test_counter == 5:
        break 

result_df

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
302
302
302
302
302
302


Exception: Failed 5 times in a row

## Testing Area

In [38]:
result_test_df = pd.DataFrame()
url_shared_app = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html" 
response = requests.get(url_shared_app, allow_redirects=False)
print(response.status_code)


soup = BeautifulSoup(response.text, 'html.parser')

# Extracting the title
title = soup.find('h1', class_='headline headline-detailed-view-title').text
title = title.replace('\n', '').strip()

# Extracting the size
size = soup.find('b', class_='key_fact_value').text
size = size.replace('\n', '').strip()

# TODO: Adding the count of people in the flat
# TODO: Maybe Restructure the replacing of the \n and strip to a function and do it in the df
# Extracting the cost panel
cost_panel = soup.find_all('div', class_='panel section_panel')[1]

rent = cost_panel.find_all('span', class_='section_panel_value')[0].text
rent = rent.replace('\n', '').strip()

extra_costs = cost_panel.find_all('span', class_='section_panel_value')[1].text
extra_costs = extra_costs.replace('\n', '').strip()

other_costs = cost_panel.find_all('span', class_='section_panel_value')[2].text
other_costs = other_costs.replace('\n', '').strip()

deposit = cost_panel.find_all('span', class_='section_panel_value')[3].text
deposit = deposit.replace('\n', '').strip()

redemption_agreement = cost_panel.find_all('span', class_='section_panel_value')[4].text
redemption_agreement = redemption_agreement.replace('\n', '').strip()



# Address and Availability Panel
address_panel = soup.find_all('div', class_='panel section_panel')[2]

address = address_panel.find_all('span', class_='section_panel_detail')[0].text
address = ' '.join(address.split())

available_from = address_panel.find_all('span', class_='section_panel_value')[0].text
available_from = available_from.replace('\n', '').strip()

# Sometimes the available till is not available, so we need to check if the second span is 'frei bis:' --> if not we need to set it to n.a.
if address_panel.find_all('span', class_='section_panel_detail')[2].text.strip() == 'frei bis:':
    available_till = address_panel.find_all('span', class_='section_panel_value')[1].text
    available_till = available_till.replace('\n', '').strip()
    online_since = address_panel.find_all('b', class_='noprint')[0].text
    online_since = online_since.replace('\n', '').strip()

else:
    available_till = "n.a."
    online_since = address_panel.find_all('b', class_='noprint')[0].text
    online_since = online_since.replace('\n', '').strip()


input_set = {'Link': url_shared_app, 'Title': title, 'Size': size, 'Rent': rent, 'Extra Costs': extra_costs, 'Other Costs': other_costs, 'Deposit': deposit, 'Redemption Agreement': redemption_agreement, 'Address': address, 'Available From': available_from, 'Available Till': available_till, 'Online Since': online_since}
result_test_df = result_test_df._append(input_set, ignore_index=True)


print(address_panel.find_all('span', class_='section_panel_detail')[2].text.strip())
result_test_df


200
frei bis:


,Link,Title,Size,Rent,Extra Costs,Other Costs,Deposit,Redemption Agreement,Address,Available From,Available Till,Online Since
0,https://www.wg-gesucht.de/wg-zimmer-in-Koeln-A...,ab 1.9. Sonniges Zimmer/ Sunny room - tempor...,9m²,300€,100€,n.a.,400€,n.a.,Bernhard-Letterhaus-Str. 0 50670 Köln Agnesvie...,01.07.2024,01.10.2024,1 Tag


## Scraperapi-Approach

In [3]:
config = configparser.ConfigParser()
config.read('local.ini')

# Extract the password
scraper_api_key = config['scraperapi']['key'].strip()
payload = {'api_key': scraper_api_key, 'url': "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html"}
r = requests.get('http://api.scraperapi.com', params=payload)
print(r.text)
print(r.status_code)


<!DOCTYPE html>
<html lang="de">
<head>
        <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width" id="viewport"/>
        <meta name="Author" content="WG-Gesucht.de"/>

            <meta name="ROBOTS" content="NOARCHIVE"/>
            
            <script data-cmp-ab="1">
            window.cmp_block_ignoredomains = [
                "stripe.com",
                "meineschufa.de",
                "my.matterport.com",
                "recaptcha.net",
                "paypal.com",
                "acrobatservices.adobe.com",
                "itsmydata.de",
                "static.elfsight.com"
            ];
        </script>
        <script type="text/javascript" data-cmp-ab="1" src="https://cdn.consentmanager.net/delivery/autoblock/15144.js" data-cmp-id="15144" data-cmp-host="a.delivery.consentmanager.net" data-cmp-cdn="cdn.consentmanager.net"></script>
    
    
                <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
        new

200